In [3]:
# RAG HW
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS 
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
  temperature = 0.1
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
  separator="\n",
  chunk_size=600,
  chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

#  이후부터 stuff로 갈지 am map 전략으로 갈지 결정해야 한다.------------------------------------------------------------------------------------------
prompt = ChatPromptTemplate.from_messages([
  ("system", "You are helpful assistant. Answer questions using onlythe following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
  ("human", "{question}")
])
chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
)
content = chain.invoke("Is Aaronson guilty?") 

AIMessage(content='Yes, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.')

In [ ]:
chain.invoke("What message did he write on the table?")

In [ ]:
chain.invoke("Who is Julia?")